## predict

In [4]:
# UltralyticsのYOLOv8を使用して画像推論を実行
from ultralytics import YOLO
import os 

model = YOLO(r"D:\prog_py\Yolo_trainer\runs\train\yolo12n-seg-pod\weights\best.pt")

In [5]:
# 指定されたパラメータで画像推論を実行
print("推論を開始します...")

results = model.predict(
    source=source_path,      # 画像フォルダ: detect_images
    imgsz=1200,              # 画像サイズ: 800
    save_txt=True,          # テキストファイル保存: True
    line_width=1,           # バウンディングボックスの線の太さ: 1
    save=True,              # 推論結果の画像を保存
    conf=0.25,              # 信頼度のしきい値
    device='0',           # デバイス（GPUが利用可能な場合は'0'に変更）
    project='runs/detect',  # 結果保存ディレクトリ
    name='yolo_detection',  # 実験名 # 既存のディレクトリがある場合上書き
    retina_masks=True,
    agnostic_nms=True,
)

print("✓ 推論が完了しました！")
print(f"結果は runs/detect/yolo_detection に保存されました")


推論を開始します...


NameError: name 'source_path' is not defined

In [6]:
# dataset.yamlを使用したモデル検証
print("=== モデル検証を開始します ===\n")

# dataset.yamlのパス
dataset_yaml_path = r"D:\Data\pods\2025_yolo_pod\labels\YOLODataset\dataset.yaml"

# dataset.yamlファイルの存在確認
if os.path.exists(dataset_yaml_path):
    print(f"✓ データセット設定ファイル確認: {dataset_yaml_path}")
    
    # ファイルの内容を表示
    with open(dataset_yaml_path, 'r', encoding='utf-8') as f:
        content = f.read()
        print(f"\nデータセット設定:")
        print(content)
else:
    print(f"✗ データセット設定ファイルが見つかりません: {dataset_yaml_path}")
    print("検証を実行できません。")


=== モデル検証を開始します ===

✓ データセット設定ファイル確認: D:\Data\pods\2025_yolo_pod\labels\YOLODataset\dataset.yaml

データセット設定:
path: \\?\D:\Data\pods\2025_yolo_pod\labels\YOLODataset
train: images/train
val: images/val
test:

names:
    0: pod



In [ ]:
# 画像サイズ（imgsz）はmax stride 32の倍数でグリッドサーチ
if os.path.exists(dataset_yaml_path):
    print("🔍 画像サイズ（32の倍数, 1500まで）で検証を実行しています...")

    # 32の倍数で500から1504まで
    grid_imgsz = list(range(512, 1024, 32))

    for imgsz in grid_imgsz:
        print(f"\n--- 画像サイズ: {imgsz} で検証中 ---")
        validation_results = model.val(
            data=dataset_yaml_path,
            imgsz=imgsz,
            plots=True,
            batch=8,
            save_json=True,
            device='0',
            project='runs/val',
            name=f'val_imgs_{imgsz}',
            verbose=True,
        )
        metrics = getattr(validation_results, 'metrics', None)
        if metrics is not None:
            map_50_95 = getattr(metrics, 'mAP_0.5_0.95', None)
            precision = getattr(metrics, 'precision', None)
            recall = getattr(metrics, 'recall', None)
            print(f"  - imgsz={imgsz}: mAP@0.5-0.95={map_50_95}, Precision={precision}, Recall={recall}")
        else:
            print(f"  - imgsz={imgsz}: メトリクス取得不可")

else:
    print("❌ データセット設定ファイルが見つからないため、検証をスキップします。")


🔍 画像サイズ（32の倍数, 1500まで）で検証を実行しています...

--- 画像サイズ: 640 で検証中 ---
Ultralytics 8.3.203  Python-3.12.6 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
val: Fast image access  (ping: 0.10.0 ms, read: 763.2153.1 MB/s, size: 169.0 KB)
val: Scanning \\?\D:\Data\pods\2025_yolo_pod\labels\YOLODataset\labels\val.cache... 13 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 13/13 12.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.1it/s 24.1s49.9s
                   all         13        554      0.714      0.625      0.664      0.379      0.725      0.621      0.665      0.316
Speed: 1.9ms preprocess, 25.8ms inference, 0.0ms loss, 167.8ms postprocess per image
Saving D:\prog_py\yolo12_detect\runs\val\val_imgs_640\predictions.json...
Results saved to D:\prog_py\yolo12_detect\runs\val\val_imgs_640
  - imgsz=640: メトリクス取得不可

--- 画像サイズ: 672 で検証中 ---
Ultralytic

KeyboardInterrupt: 

##　出力変換

In [1]:
from ultralytics import YOLO

# Load a model

model = YOLO(r"D:\prog_py\Yolo_trainer\runs\train\train9\weights\best.pt")  # load a custom trained model

# Export the model
model.export(format="onnx",
             imgsz=1200)

Ultralytics 8.3.203  Python-3.12.6 torch-2.6.0+cu124 CPU (Intel Core(TM) i7-10875H 2.30GHz)
WARNING imgsz=[1200] must be multiple of max stride 32, updating to [1216]
YOLO11l-seg summary (fused): 203 layers, 27,585,363 parameters, 0 gradients, 131.8 GFLOPs

PyTorch: starting from 'D:\prog_py\Yolo_trainer\runs\train\train9\weights\best.pt' with input shape (1, 3, 1216, 1216) BCHW and output shape(s) ((1, 37, 30324), (1, 32, 304, 304)) (53.4 MB)

ONNX: starting export with onnx 1.19.1 opset 19...
ONNX: slimming with onnxslim 0.1.72...
ONNX: export success  7.0s, saved as 'D:\prog_py\Yolo_trainer\runs\train\train9\weights\best.onnx' (106.0 MB)

Export complete (12.1s)
Results saved to D:\prog_py\Yolo_trainer\runs\train\train9\weights
Predict:         yolo predict task=segment model=D:\prog_py\Yolo_trainer\runs\train\train9\weights\best.onnx imgsz=1216  
Validate:        yolo val task=segment model=D:\prog_py\Yolo_trainer\runs\train\train9\weights\best.onnx imgsz=1216 data=D:\prog_py\Yolo_

'D:\\prog_py\\Yolo_trainer\\runs\\train\\train9\\weights\\best.onnx'

## auto label

In [4]:
from ultralytics.data.annotator import auto_annotate

auto_annotate(data=r"Dataset\insta360\train",
              det_model=r"D:\prog_py\Yolo_trainer\runs\train\train9\weights\best.pt",
              imgsz=1024,
              sam_model="sam2.1_l.pt")


image 1/69 d:\prog_py\yolo12_detect\Dataset\insta360\train\024_01_b.jpg: 1024x576 84 pods, 763.0ms


KeyboardInterrupt: 